In [ ]:
import os
from keras import regularizers
from keras.layers import Dense, Input
from keras.models import Model
import mne
import numpy as np
raw_dir = 'D:\\NING - spindle\\training set\\'
os.chdir(raw_dir)
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
raw_names = ['suj11_l2nap_day2.fif','suj11_l5nap_day1.fif',
             'suj12_l2nap_day1.fif','suj12_l5nap_day2.fif',
             'suj13_l2nap_day2.fif','suj13_l5nap_day1.fif',
             'suj14_l2nap_day2.fif','suj14_l5nap_day1.fif',
             'suj15_l2nap_day1.fif','suj15_l5nap_day2.fif',
             'suj16_l2nap_day2.fif','suj16_l5nap_day1.fif']
epochs = []
for ii,raw_name in enumerate(raw_names):
    raw = mne.io.read_raw_fif(raw_name)
    duration = 5 # 5 seconds
    events = mne.make_fixed_length_events(raw,id=1,duration=duration,)
    epoch=mne.Epochs(raw,events,tmin=0,tmax=duration,preload=True,
                            proj=False).resample(128)
    epoch=epoch.pick_channels(epoch.ch_names[:-2])
    if ii == 0:
        epochs = epoch.get_data()
    else:
        epochs = np.concatenate([epochs,epoch.get_data()])
    raw.close()
    

In [ ]:
epochs.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test = train_test_split(epochs)
x_train = x_train.astype('float32').reshape(x_train.shape[0],-1)
x_train /= np.max(x_train)
x_test  = x_test.astype('float32').reshape(x_test.shape[0],-1)
x_test /= np.max(x_test)

In [ ]:
input_shape = x_train.shape[1]

In [ ]:
# this is the size of our encoded representations
encoding_dim = 32  

input_img = Input(shape=(input_shape,))
encoded = Dense(256, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_img)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)

decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(input_shape, activation='sigmoid')(decoded)

autoencoder = Model(input=input_img, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_img, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input_1 = Input(shape=(64,))
encoded_input_2 = Input(shape=(128,))
encoded_input_3 = Input(shape=(256,))

# retrieve the last layer of the autoencoder model
decoder_layer_1 = autoencoder.layers[-3]
decoder_layer_2 = autoencoder.layers[-2]
decoder_layer_3 = autoencoder.layers[-1]

# create the decoder model
decoder_1 = Model(input = encoded_input_1, output = decoder_layer_1(encoded_input_1))
decoder_2 = Model(input = encoded_input_2, output = decoder_layer_2(encoded_input_2))
decoder_3 = Model(input = encoded_input_3, output = decoder_layer_3(encoded_input_3))

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=2000,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
#a=autoencoder.predict(x_test)
n = 1
fig, ax = plt.subplots(nrows=2,figsize=(8,8))
_=ax[0].plot(x_test[n].reshape(61,640))
_=ax[1].plot(a[n].reshape(61,640))